# Data Preparation

This notebook will


1. Annotate the dataset (Sleep 0 /Awake 1)
2. Signal Preparation (scaling, missing data, outliers, smoothing)
3. Subset generation (light, medium, heavy)

## Import


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl

In [ ]:
df_signals = pl.read_parquet("data/train_series.parquet")

In [ ]:
df_events = pd.read_csv("data/train_events.csv")

**Convert timestamp to datetime**


In [ ]:
df_signals = df_signals.with_columns(
    pl.col("timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%z")
)

In [ ]:
df_events.timestamp = pd.to_datetime(df_events.timestamp)

**Min-max normalization**

In [ ]:
# min-max scaling
cols_to_scale = ["anglez", "enmo"]
new_cols = [f"{col}_scaled" for col in cols_to_scale]

df_signals[cols_to_scale] = df_signals.select(cols_to_scale).with_columns(
    (pl.col(cols_to_scale) - pl.col(cols_to_scale).min()) / (pl.col(cols_to_scale).max() - pl.col(cols_to_scale).min())
)

**Annotation Sleep // Awake**

In [ ]:
# Your code here ...

**Missing Data**

Remove signals 6 hours after awake and 6 hours before sleep when an annotation is missing

In [ ]:
# Your code here ...

**Smoothing**

In [ ]:
# Your code here ...

**Stratified Export**

In [ ]:
# Your code here ...